In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, string, random
from datetime import datetime
from omegaconf import OmegaConf
import wandb
import torch
import lightning as L
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from monai.networks.nets import UNet

sys.path.append('../')
from data_utils import MNMv2DataModule
from unet import LightningSegmentationModel



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

nnUNet_raw_data_base is not defined and nnU-Net can only be used on data for which preprocessed files are already present on your system. nnU-Net cannot be used for experiment planning and preprocessing like this. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up properly.
nnUNet_preprocessed is not defined and nnU-Net can not be used for preprocessing or training. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up.
RESULTS_FOLDER is not defined and nnU-Net cannot be used for training or inference. If this is not in

In [3]:
# load configs
mnmv2_config   = OmegaConf.load('../../configs/mnmv2.yaml')
unet_config    = OmegaConf.load('../../configs/monai_unet.yaml')
trainer_config = OmegaConf.load('../../configs/unet_trainer.yaml')

In [4]:
# init datamodule
datamodule = MNMv2DataModule(
    data_dir=mnmv2_config.data_dir,
    vendor_assignment=mnmv2_config.vendor_assignment,
    batch_size=mnmv2_config.batch_size,
    binary_target=mnmv2_config.binary_target,
    non_empty_target=mnmv2_config.non_empty_target,
)

In [6]:
datamodule.setup(stage='fit')

train = datamodule.train_dataloader()
print(len(train.generator._data))

4551


In [5]:
from torch.utils.data import Dataset
class MNMv2Subset(Dataset):
    def __init__(
        self,
        input,
        target,
        # weight
    ):
        self.input = input
        self.target = target
        # self.weight = weight

    def __len__(self):
        return self.input.shape[0]
    
    def __getitem__(self, idx):
        return {
            "input": self.input[idx], 
            "target": self.target[idx],
        }

datamodule.setup(stage='test')  # Получаем доступ к тестовому набору
test_inputs = torch.stack([sample["input"] for sample in datamodule.mnm_test])
test_targets = torch.stack([sample["target"] for sample in datamodule.mnm_test])

train_dataset_from_test = MNMv2Subset(
    input=test_inputs,
    target=test_targets
)

# Заменяем обучающий набор на новый датасет из тестовых данных
datamodule.mnm_train = train_dataset_from_test

In [6]:
# init model

cfg = OmegaConf.create({
    'unet_config': unet_config,
    'binary_target': True if unet_config.out_channels == 1 else False,
    'lr': unet_config.lr,
    'patience': unet_config.patience,
    'dataset': OmegaConf.to_container(mnmv2_config),
    'unet': OmegaConf.to_container(unet_config),
    'trainer': OmegaConf.to_container(trainer_config)
})

model = LightningSegmentationModel(cfg=cfg)
print(model)


LightningSegmentationModel(
  (model): UNet(
    (model): Sequential(
      (0): ResidualUnit(
        (conv): Sequential(
          (unit0): Convolution(
            (conv): Conv2d(1, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
            (adn): ADN(
              (N): InstanceNorm2d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
              (D): Dropout(p=0.0, inplace=False)
              (A): PReLU(num_parameters=1)
            )
          )
          (unit1): Convolution(
            (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (adn): ADN(
              (N): InstanceNorm2d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
              (D): Dropout(p=0.0, inplace=False)
              (A): PReLU(num_parameters=1)
            )
          )
          (unit2): Convolution(
            (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (adn): ADN(
        

AttributeError: 'MultiThreadedAugmenter' object has no attribute 'data_loader'

In [16]:
# print(model.model.model[:2])
for param in model.model.model[:2].parameters():
    param.requires_grad = False

# Проверить, что заморожено
for name, param in model.named_parameters():
    print(f"{name}: requires_grad = {param.requires_grad}")

model.model.0.conv.unit0.conv.weight: requires_grad = False
model.model.0.conv.unit0.conv.bias: requires_grad = False
model.model.0.conv.unit0.adn.A.weight: requires_grad = False
model.model.0.conv.unit1.conv.weight: requires_grad = False
model.model.0.conv.unit1.conv.bias: requires_grad = False
model.model.0.conv.unit1.adn.A.weight: requires_grad = False
model.model.0.conv.unit2.conv.weight: requires_grad = False
model.model.0.conv.unit2.conv.bias: requires_grad = False
model.model.0.conv.unit2.adn.A.weight: requires_grad = False
model.model.0.conv.unit3.conv.weight: requires_grad = False
model.model.0.conv.unit3.conv.bias: requires_grad = False
model.model.0.conv.unit3.adn.A.weight: requires_grad = False
model.model.0.residual.weight: requires_grad = False
model.model.0.residual.bias: requires_grad = False
model.model.1.submodule.0.conv.unit0.conv.weight: requires_grad = False
model.model.1.submodule.0.conv.unit0.conv.bias: requires_grad = False
model.model.1.submodule.0.conv.unit0.a

In [7]:
# infered variable
patience = unet_config.patience * 2

now = datetime.now()
filename = 'mnmv2-' + now.strftime("%H-%M_%d-%m-%Y")

# init trainer
if trainer_config.logging:
    wandb.finish()
    logger = WandbLogger(
        project="lightning", 
        log_model=True, 
        name=filename
    )
else:
    logger = None

# trainer
trainer = L.Trainer(
    limit_train_batches=trainer_config.limit_train_batches,
    max_epochs=trainer_config.max_epochs,
    logger=logger,
    callbacks=[
        EarlyStopping(
            monitor=trainer_config.early_stopping.monitor, 
            mode=trainer_config.early_stopping.mode, 
            patience=patience
        ),
        ModelCheckpoint(
            dirpath=trainer_config.model_checkpoint.dirpath,
            filename=filename,
            save_top_k=trainer_config.model_checkpoint.save_top_k, 
            monitor=trainer_config.model_checkpoint.monitor,
        )
    ],
    precision='16-mixed',
    gradient_clip_val=0.5,
    devices=[0]
)

Using 16bit Automatic Mixed Precision (AMP)
/home/mikhelson/MedImSeg-Lab24/CLUE/lib/python3.8/site-packages/lightning/pytorch/plugins/precision/amp.py:52: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [8]:
trainer.fit(model, datamodule=datamodule)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: 1069035f (rivman). Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name  | Type       | Params | Mode 
---------------------------------------------
0 | model | UNet       | 794 K  | train
1 | loss  | DiceCELoss | 0      | train
---------------------------------------------
794 K     Trainable params
0         Non-trainable params
794 K     Total params
3.178     Total estimated model params size (MB)


Epoch 99: 100%|██████████| 50/50 [00:12<00:00,  4.07it/s, v_num=rmiy]      

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 50/50 [00:12<00:00,  4.05it/s, v_num=rmiy]


In [ ]:
datamodule.setup(stage='test')
# model = model.to(device)
model.eval()
test_perf = trainer.test(model, datamodule=datamodule)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Testing DataLoader 0: 100%|██████████| 40/40 [00:03<00:00, 11.75it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_dsc            0.7333963513374329
        test_loss           0.5420395135879517
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


: 

In [9]:
checkpoint_path = '../../checkpoints/mnmv2-11-52_29-10-2024.ckpt'

load_as_lightning_module = False
load_as_pytorch_module = True

if load_as_lightning_module:
    unet_config    = OmegaConf.load('../../configs/monai_unet.yaml')
    unet = UNet(
        spatial_dims=unet_config.spatial_dims,
        in_channels=unet_config.in_channels,
        out_channels=unet_config.out_channels,
        channels=[unet_config.n_filters_init * 2 ** i for i in range(unet_config.depth)],
        strides=[2] * (unet_config.depth - 1),
        num_res_units=4
    )
    model = LightningSegmentationModel.load_from_checkpoint(
        checkpoint_path,
        map_location=torch.device("cpu"),
        model=unet,
        binary_target=True if unet_config.out_channels == 1 else False,
        lr=unet_config.lr,
        patience=unet_config.patience,
        # cfg=OmegaConf.to_container(unet_config)
    )

elif load_as_pytorch_module:
    checkpoint = torch.load(checkpoint_path, map_location=torch.device("cpu"))
    model_state_dict = checkpoint['state_dict']
    model_state_dict = {k.replace('model.model.', 'model.'): v for k, v in model_state_dict.items() if k.startswith('model.')}
    model_config = checkpoint['hyper_parameters']['cfgs']

    print(model_config)

    unet = UNet(
        spatial_dims=model_config['unet']['spatial_dims'],
        in_channels=model_config['unet']['in_channels'],
        out_channels=model_config['unet']['out_channels'],
        channels=[model_config['unet']['n_filters_init'] * 2 ** i for i in range(model_config['unet']['depth'])],
        strides=[2] * (model_config['unet']['depth'] - 1),
        num_res_units=4
    )

    unet.load_state_dict(model_state_dict)

/tmp/ipykernel_10508/3995855781.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=torch.device("cpu"))


{'dataset': {'data_dir': '../../../../../data/MNM/', 'vendor_assignment': {'train': 'siemens', 'test': 'ge'}, 'batch_size': 32, 'binary_target': False, 'non_empty_target': False}, 'unet': {'n_filters_init': 16, 'depth': 4, 'spatial_dims': 2, 'in_channels': 1, 'out_channels': 4, 'num_res_units': 4, 'lr': 0.001, 'patience': 5}, 'trainer': {'train_transforms': 'global_transforms', 'limit_train_batches': 50, 'max_epochs': 100, 'early_stopping': {'monitor': 'val_loss', 'mode': 'min'}, 'model_checkpoint': {'save_top_k': 2, 'dirpath': '../../pre-trained/monai-unets', 'monitor': 'val_loss'}, 'logging': True}}


In [10]:
model.cfg

{'unet_config': {'n_filters_init': 16, 'depth': 4, 'spatial_dims': 2, 'in_channels': 1, 'out_channels': 4, 'num_res_units': 4, 'lr': 0.001, 'patience': 5, 'clue_softmax_t': 1, 'batch_size': 32, 'binary_target': False}, 'binary_target': False, 'lr': 0.001, 'patience': 5, 'dataset': {'data_dir': '/home/mikhelson/MedImSeg-Lab24/data/MNM/', 'vendor_assignment': {'train': 'siemens', 'test': 'ge'}, 'batch_size': 32, 'binary_target': False, 'non_empty_target': False}, 'unet': {'n_filters_init': 16, 'depth': 4, 'spatial_dims': 2, 'in_channels': 1, 'out_channels': 4, 'num_res_units': 4, 'lr': 0.001, 'patience': 5, 'clue_softmax_t': 1, 'batch_size': 32, 'binary_target': False}, 'trainer': {'train_transforms': 'global_transforms', 'limit_train_batches': 50, 'max_epochs': 100, 'early_stopping': {'monitor': 'val_loss', 'mode': 'min'}, 'model_checkpoint': {'save_top_k': 2, 'dirpath': '../../MedImSeg-Lab24/pre-trained/trained_UNets', 'monitor': 'val_loss'}, 'logging': True}}